In [107]:
import json
import pathlib

package_paths = {
    "ips": pathlib.Path(".") / "packages" / "hl7.fhir.uv.ips-1.1.0",
    "zib2017": pathlib.Path(".") / "packages" / "nictiz.fhir.nl.stu3.zib2017-2.2.20",
    "zib2020": pathlib.Path(".") / "packages" / "nictiz.fhir.nl.r4.zib2020-0.11.0-beta.1"
}

class Profile:
    def __init__(self, package, profile_name):
        self.package = package
        self.name = profile_name
        
        if package == "ips":
            file = package_paths[package] / f"StructureDefinition-{profile_name}.json"
        else:
            file = package_paths[package] / f"{profile_name}.json"

        with open(file) as fd:
            content = json.load(fd)

        # Fish out interesting data: min and max cardinality and binding strengths
        self.elements = {}
        for el in content["differential"]["element"]:
            path = el["path"]
            data = {}
            if "min" in el:
                data["min"] = el["min"]
            if "max" in el:
                data["max"] = el["max"]
            if "binding" in el:
                if "strength" in el["binding"]:
                    data["binding_strength"] = el["binding"]["strength"]
                if "valueSet" in el["binding"]:
                    data["valueSet"] = el["binding"]["valueSet"].split("|")[0]
                if "valueSetReference" in el["binding"]: # Happens when we have extensions in the valueSet
                    data["valueSet"] = el["binding"]["valueSetReference"]["reference"].split("|")[0]
            
            # If we found something of interest, complete all data with the defaults from the snapshot so we can make
            # a comparison
            if len(data):
                snapshot_el = list(filter(lambda e: e["path"] == path, content["snapshot"]["element"]))[0]
                if "min" not in data:
                    data["min"] = snapshot_el["min"]
                if "max" not in data:
                    data["max"] = snapshot_el["max"]
                if "binding" in snapshot_el:
                    if "binding_strength" not in data:
                        data["binding_strength"] = snapshot_el["binding"]["strength"]
                    if "valueSet" not in data:
                        if "valueSetReference" in snapshot_el:
                            data["valueSet"] = snapshot_el["binding"]["valueSetReference"]["reference"].split("|")[0]
                        else:
                            data["valueSet"] = snapshot_el["binding"]["valueSet"].split("|")[0]
                self.elements[path] = data

    def fits_in(self, other):
        findings = []
        for path in self.elements:
            if path in other.elements:
                self_el  = self.elements[path]
                other_el = other.elements[path]

                if "min" in self_el and "min" in other_el:
                    if other_el["min"] > self_el["min"]:
                        findings.append(f"{path}: required in {other.name} but optional in {self.name}")

                if "max" in self_el and "max" in other_el:
                    if other_el["max"] != "*": # if other is unrestricted we always fit
                        if self_el["max"] == "*":
                            findings.append(f"{path}: restricted to {other_el["max"]} in {other.name} but unrestricted in {self.name}")
                        else:
                            self_max = int(self_el["max"])
                            other_max = int(other_el["max"])
                            if self_max > other_max:
                                findings.append(f"{path}: restricted to {other_max} in {other.name} but to {self_max} in {self.name}")
                
                if "binding_strength" in self_el:
                    if self_el["binding_strength"] == "required":
                        if other_el["binding_strength"] == "required":
                            if self_el["valueSet"] != other_el["valueSet"]:
                                findings.append(f"{path}: One of the profiles has a more restricted value set than the other. Manual check needed.")
                        else:
                            if self_el["valueSet"] != other_el["valueSet"]:
                                findings.append(f"{path}: terminology is unrestricted in {other.name} but restricted in {self.name}. Manual check needed.")
                            else:
                                # More restricted on the same ValueSet, so it fits
                                pass
                    elif self_el["binding_strength"] == "extensible":
                        if self_el["valueSet"] != other_el["valueSet"]:
                            findings.append(f"{path}: other ValueSet, bound extensible, in {self.name}. Manual check needed.")
                        else:
                            # More restricted on the same ValueSet, so it fits
                            pass
                    elif other_el["binding_strength"] == "extensible" or other_el["binding_strength"] == "required":
                        findings.append(f"{path}: ValueSet in {self.name} is less restricted than in {other.name}.")
                    elif self_el["valueSet"] != other_el["valueSet"]:
                        findings.append(f"{path}: ValueSet preference in {other.name} is different than in {self.name}.")

        if len(findings):
            print(f"{self.name} ({self.package}) does not seem to fit in {other.name} ({other.package})"),
            for finding in findings:
                print(f"* {finding}")
                
zib_Problem2017 = Profile("zib2017", "zib-Problem")
zib_Problem2020 = Profile("zib2020", "zib-Problem")
Condition_uv_ips = Profile("ips", "Condition-uv-ips")
zib_Problem2017.fits_in(Condition_uv_ips)
Condition_uv_ips.fits_in(zib_Problem2017)
zib_Problem2020.fits_in(Condition_uv_ips)
Condition_uv_ips.fits_in(zib_Problem2020)

zib-Problem (zib2017) does not seem to fit in Condition-uv-ips (ips)
* Condition.category: other ValueSet, bound extensible, in zib-Problem. Manual check needed.
* Condition.code: other ValueSet, bound extensible, in zib-Problem. Manual check needed.
* Condition.bodySite: other ValueSet, bound extensible, in zib-Problem. Manual check needed.
Condition-uv-ips (ips) does not seem to fit in zib-Problem (zib2017)
* Condition.category: ValueSet in Condition-uv-ips is less restricted than in zib-Problem.
* Condition.code: ValueSet in Condition-uv-ips is less restricted than in zib-Problem.
* Condition.bodySite: ValueSet in Condition-uv-ips is less restricted than in zib-Problem.
zib-Problem (zib2020) does not seem to fit in Condition-uv-ips (ips)
* Condition.clinicalStatus: required in Condition-uv-ips but optional in zib-Problem
* Condition.category: terminology is unrestricted in Condition-uv-ips but restricted in zib-Problem. Manual check needed.
* Condition.code: required in Condition-uv